# RAG system backend

In [ ]:
import random
import time
import os
import pandas as pd

from openai import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
doc_dir = "docs"

In [ ]:
! ls docs

In [ ]:
client = OpenAI() 

In [ ]:
def load_and_index_documents(doc_dir):
    texts = []

    for filename in os.listdir(doc_dir):
        file_path = os.path.join(doc_dir, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            texts.append(content)

    text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=200)
    documents = [Document(page_content=t) for t in texts]
    chunks = text_splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings() #openai_api_key=OPENAI_API_KEY
    vector_store = FAISS.from_documents(chunks, embeddings)

    return vector_store

In [ ]:
# Search the index for relevant information
def search_documents(query, vector_store):
    search_results = vector_store.similarity_search(query, k=5)  
    context = "\n".join([doc.page_content for doc in search_results])
    return context


def response_generator(prompt, context):
    messages = [
        {"role":"system", "content":f"retrieved context: {context}"},
        {"role":"user", "content":prompt},
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=False
    )


    return response.choices[0].message.content if response.choices[0] else None

In [ ]:
vector_store = load_and_index_documents(doc_dir)

In [ ]:
questions = [
    "What is Evidently?",
    "How do I get a Data Drift report?",
    "Are LLMs suppirted in Evidently?"
]

In [ ]:
answers = [response_generator(question, search_documents(question, vector_store)) for question in questions]

In [ ]:
for answer in answers:
    print(answer)
    print("\n")

# RAG evaluation

In [ ]:
from evidently.ui.workspace.cloud import CloudWorkspace
from evidently.future.datasets import Dataset
from evidently.future.datasets import DataDefinition
from evidently.future.datasets import Descriptor
from evidently.future.descriptors import *
from evidently.future.report import Report
from evidently.future.presets import TextEvals
from evidently.future.metrics import *
from evidently.future.tests import *

### connect to the cloud

In [ ]:
API_KEY = 'YOUR KEY HERE'

### dataset-based evaluation

In [ ]:
ws = CloudWorkspace(token=API_KEY, url="https://app.evidently.cloud")

In [ ]:
project = ws.get_project("PROJECT ID")

In [ ]:
test_df_full = ws.load_dataset(dataset_id = "DATASET_ID")

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
test_df_full.head()

In [ ]:
test_df_full["context"] = [
    search_documents(question, vector_store) for question in test_df_full["questions"]
]

In [ ]:
test_df_full["generated_answers"] = [
    response_generator(test_df_full.loc[i, "questions"], test_df_full.loc[i, "context"]) for i in range(test_df_full.shape[0])
]

In [ ]:
test_df_full.head()

In [ ]:
context_based_evals = Dataset.from_pandas(
    pd.DataFrame(test_df_full),
    data_definition=DataDefinition(
        text_columns=["questions", "answers", "generated_answers", "context"],
    ),
    descriptors=[
        CorrectnessLLMEval("generated_answers", target_output="answers"),
        FaithfulnessLLMEval("generated_answers", context="context"),
    ]
)
context_based_evals.as_dataframe()

In [ ]:
report = Report([
    TextEvals()
])

my_eval = report.run(context_based_evals, None)
my_eval

In [ ]:
ws.add_run(project.id, my_eval, include_data=True)